In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive/Colab Notebooks/kobo2020'
%ls


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks/kobo2020
AttentionSeq2seq.pkl                negative_sampling_layer.py
attention_seq2seq.py                neko.txt
BetterRnnlm.pkl                     nlp100_chap9
better_rnnlm.py                     nlp100_chap9.pkl
cbow_params.pkl                     nlp100_chap9.txt
cbow.py                             __pycache__/
ch04/                               PyTorchTutorial1.ipynb
ch06/                               repo10.ipynb
ch07/                               repo10test.ipynb
ch07＿pra.ipynb                     repo11_test.ipynb
ch08/                               repo6.ipynb
ch08_pra.ipynb                      repo7.ipynb
common/                             repo8.ipynb
data70/                             repo9.ipynb
dataset/                            simple_rnnlm.py
enword2id.pickle                    small_paralle

In [ ]:
 #データのロード
 !git clone https://github.com/odashi/small_parallel_enja.git

Cloning into 'small_parallel_enja'...
remote: Enumerating objects: 35, done.
remote: Total 35 (delta 0), reused 0 (delta 0), pack-reused 35
Unpacking objects: 100% (35/35), done.


In [ ]:
#seq2seq2data.py　データを読み込んでIDを付けたものを保存
import re
from collections import Counter
import pickle

min_freq = 2

# 日本語
ja_path = "small_parallel_enja/train.ja"
word2id_path = "jaword2id.pickle"

word_list = []
with open(ja_path) as f:
    for line in f:
        if not line:
            continue
        for word in line.split():
            word_list.append(word)

word_count = Counter(word_list).most_common()

word2id = {"[BOS]":1, "[EOS]":0, "[UNK]":2} # BOS:英語文の開始を意味するトークン， EOS：英語文の終端を意味するトークン， UNK：未知語を意味するトークン
index = 3
for word, freq in word_count:
    if freq < min_freq:
        pass # 規定回数未満は登録しない
    else:
        word2id[word] = index
        index += 1

# 保存
pickle.dump(word2id, open(word2id_path, "wb"))
# 表示 単語の数
print("word_num", len(word2id), word2id)


#英語
en_path = "small_parallel_enja/train.en"
word2id_path ="enword2id.pickle"

word_list = []
with open(en_path) as f:
    for line in f:
        if not line:
            continue
        for word in line.split():
            word_list.append(word)

word_count = Counter(word_list).most_common()

word2id = {"[BOS]":1, "[EOS]":0, "[UNK]":2} # BOS:英語文の開始を意味するトークン， EOS：英語文の終端を意味するトークン， UNK：未知語を意味するトークン
index = 3
for word, freq in word_count:
    if freq < min_freq:
        pass # 規定回数未満は登録しない
    else:
        word2id[word] = index
        index += 1

# 保存
pickle.dump(word2id, open(word2id_path, "wb"))
# 表示 英単語の数
print("word_num", len(word2id), word2id)

word_num 4905 {'[BOS]': 1, '[EOS]': 0, '[UNK]': 2, '。': 3, 'は': 4, 'い': 5, 'に': 6, 'た': 7, 'を': 8, 'の': 9, 'て': 10, 'で': 11, 'な': 12, 'が': 13, '彼': 14, 'し': 15, 'る': 16, '私': 17, 'す': 18, 'っ': 19, 'ま': 20, 'か': 21, 'だ': 22, '彼女': 23, 'と': 24, 'う': 25, 'その': 26, 'も': 27, 'く': 28, 'ん': 29, '、': 30, 'あ': 31, 'ら': 32, 'こと': 33, 'よ': 34, 'この': 35, 'き': 36, 'り': 37, 'する': 38, 'お': 39, 'せ': 40, 'あなた': 41, 'かっ': 42, '行': 43, '君': 44, '何': 45, 'たち': 46, 'から': 47, 'それ': 48, '言': 49, '人': 50, 'さ': 51, 'ば': 52, 'れ': 53, 'よう': 54, 'でき': 55, 'へ': 56, 'しょ': 57, '本': 58, 'つ': 59, '話': 60, 'くださ': 61, '見': 62, 'くれ': 63, '家': 64, 'ろ': 65, '仕事': 66, 'どう': 67, 'これ': 68, 'だっ': 69, '時': 70, 'いつ': 71, 'や': 72, 'もう': 73, '思': 74, 'とても': 75, '車': 76, '知': 77, 'まで': 78, 'なさ': 79, '日': 80, '会': 81, 'ここ': 82, '時間': 83, '来': 84, 'もの': 85, '好き': 86, 'わ': 87, 'すぐ': 88, 'ね': 89, '今': 90, '誰': 91, 'そう': 92, '中': 93, '出': 94, '自分': 95, '下さ': 96, '英語': 97, '金': 98, '父': 99, 'み': 100, '一': 101, '今日': 102, '達': 103, '僕': 1

In [ ]:
#seq2seq
import pickle
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence, pad_sequence, pack_padded_sequence

In [76]:
#seq2seq
import pickle
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence, pad_sequence, pack_padded_sequence

ja_word2id_path = "jaword2id.pickle"
en_word2id_path = "enword2id.pickle"
ja_word2id = pickle.load(open(ja_word2id_path, "rb"))
en_word2id = pickle.load(open(en_word2id_path, "rb"))

#言語に応じたデータを取り出してidのリストを返す
def get_ids(word_list, lang):
    if lang == "ja":
        word2id = ja_word2id
    else:
        word2id = en_word2id

    ids = []
    for word in word_list:
        if word in word2id:
            ids.append(word2id[word])
        else:
            ids.append(word2id["[UNK]"])
    return ids


# 使用するデータの英語と日本語の文に応じたid列のテンソル が入ったリストを返す
def get_data(ja_fname, en_fname):
    ja_ids_list = []
    en_ids_list = []

    with open(ja_fname) as f:
        for line in f:
            if not line:
                continue
            word_list = line.split()
            ids = get_ids(word_list, "ja")
            ja_ids_list.append(torch.tensor(ids))

    with open(en_fname) as f:
        for line in f:
            if not line:
                continue
            word_list = line.split()
            ids = get_ids(word_list, "en")
            en_ids_list.append(torch.tensor(ids))

    return ja_ids_list, en_ids_list

class Seq2SeqRNN(nn.Module):
  
    def __init__(self, ja_vocab_size, en_vocab_size, embedding_dim, hidden_dim):
        super(Seq2SeqRNN, self).__init__()
        self.ja_embedding = nn.Embedding(ja_vocab_size, embedding_dim)
        self.en_embedding = nn.Embedding(en_vocab_size, embedding_dim)
        self.encoder = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.decoder = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, en_vocab_size, bias=True)
        self.crossentropy = nn.CrossEntropyLoss()
        #print(ja_vocab_size)#4905
        #print(en_vocab_size)#4108

    def forward(self, ja_data, ja_data_len, en_data, en_data_len):
            # ja_dataを単語埋め込み
            embed_ja_data = self.ja_embedding(ja_data)
            # embed_ja_dataをPackedSequence型のデータに変換
            x = pack_padded_sequence(embed_ja_data, ja_data_len, batch_first=True, enforce_sorted=False)
            # encoderに入力し，エンコード結果を得る
            _, (h, c) = self.encoder(x)

            # en_dataの各系列の先頭に[BOS]のidを追加したbos_en_dataを作成
            ones = torch.ones(len(en_data_len), 1, dtype=torch.long).to(device)
            bos_en_data = torch.cat([ones, en_data], dim=-1)

            # en_data_lenをその分すべて1ずつ増やす
            # todo
            en_data_len += 1

            # bos_en_dataを単語埋め込み
            embed_bos_en_data = self.en_embedding(en_data)# todo

            # bos_en_dataをpack_padded_sequenceを用いてPackedSequence型のデータに変換
            packed_en_data = pack_padded_sequence(embed_bos_en_data, ja_data_len, batch_first=True, enforce_sorted=False)# todo
            # bos_en_dataのPackedSequenceとja_dataのエンコード結果を用いてデコードし，デコード結果をoutputに代入．
            # todo
            output,(dh,dc) = self.decoder(packed_en_data , (h,c))

            # デコード結果outputを線形層でアフィン変換
            pred = self.linear(output.data)# todo

            # ここからはクロスエントロピー損失計算のための正解の準備
            # en_dataの各系列の末尾に[EOS]のidを追加したeos_en_dataを作成
            zeros = torch.zeros(len(en_data_len), 1, dtype=torch.long).to(device)# todo
            eos_en_data =  torch.cat([en_data,zeros],dim = 1)# todo

            # eos_en_data(埋め込み前)をすでに1増やされたen_data_lenを用いてpack_padded_sequence．これにより単語ID列 + [EOS]のID(0)の系列のPackedSequenceが得られる
            target =  pack_padded_sequence(eos_en_data, ja_data_len, batch_first=True, enforce_sorted=False)# todo
            #print(pred.data)#grad_fnない
            loss = self.crossentropy(pred,target.data)# todo
            #print(loss)
            return loss


     # todo 日本語文を受け取り，英語文を生成するメソッド

    def generate(self, ja_data, ja_data_len):
        sampled = []
        sample_id = ja_data[0]
        ja_data = ja_data.to(device)

        for _ in range(ja_data_len):
            x = torch.tensor([sample_id]).reshape((1, 1)).to(device)

            out = self.en_embedding.forward(x)#埋め込みベクトルに変換
            _,(h,c) = self.encoder.forward(out)#隠れ状態を得る 
            out,(dh,dc) = self.decoder(out , (h,c))#デコーダーに入れる
            score = self.linear(out)#線形変換
            
            sample_id = torch.max(score.flatten(),axis = 0).indices.item()
            sampled.append(sample_id)
      
        return sampled  
"""    
     def generate(self, h, start_id, sample_size):
        sampled = []
        sample_id = start_id
        self.lstm.set_state(h)

        for _ in range(sample_size):
            x = np.array(sample_id).reshape((1, 1))
            out = self.embed.forward(x)
            out = self.lstm.forward(out)
            score = self.affine.forward(out)

            sample_id = np.argmax(score.flatten())
            sampled.append(int(sample_id))

        return sampled          


        ここで実行する手順
        1. エンコーダーLSTM層に日本語の単語埋め込み系列を入力し，各系列の最後の入力を行った時点の隠れ状態+記憶セルを取得
        2. デコーダーLSTM層に先頭に[BOS](単語idは1)を付加した英語の単語埋め込み系列とエンコーダーから得られた隠れ状態+記憶セルを入力し，各単語を入力した際の隠れ状態を得る
        3. 得られた隠れ状態を線形層に入力し，各時点における出力英単語の分布を得る
        4. 末尾に[EOS](単語idは0)を付加した英語の単語系列を正解とし，出力英単語の分布からCrossEntropyLossを計算
        5. CrossEntropyLossを返り値として返す
        具体例：エンコーダーには「これ は ペン です」の単語埋め込み系列を入力し，「です」を入力した時点の隠れ状態+記憶セルを取得．
        　　　　デコーダーには「[BOS] This is a pen」の単語埋め込み系列とエンコーダーから得た隠れ状態+記憶セルを入力し，「This is a pen [EOS]」と出力するように損失を計算
        備考：Teacher Forcingによる学習 (参考：https://satopirka.com/2018/02/encoder-decoder%E3%83%A2%E3%83%87%E3%83%AB%E3%81%A8teacher-forcingscheduled-samplingprofessor-forcing/)
​
        ヒント：
        　・[BOS]と[EOS]の付加について：0のみ，1のみが要素の任意の形状のtensorを作成できるtorch.zeros, torch.onesメソッドを使用して作成したtensorをtorch.catメソッドで単語id系列のtensorとくっつける
        　・デコーダーの出力と正解文の間の損失の計算：PackedSequence型データをLSTMに入力した場合の各時刻の隠れ状態はPackedSequence型で返ってくる．インスタンス変数「.data」でPackedSequenceの持つtensorを得られるが，
        　　　                                    PackedSequenceの作成時に入力したtensorにおける系列の並び・形状と「.data」で得られるtensorの系列の並び・形状は異なる．
        　　　　　　　　　　　　　　　　　　　　　　(参考：sequence.pyのpad_inputsと，pack_padded_sequenceによって得られるxの内容の比較)
                　　　                            そこで，正解文のtensorも一旦PackedSequence化し，tensorの並び・形状を揃えた上で「.data」でtensorを取り出し，CrossEntropyLossを計算する

"""





class S2SDataset(torch.utils.data.Dataset):
    def __init__(self, ja_data, en_data):
        self.ja_length = [len(words) for words in ja_data]
        self.en_length = [len(words) for words in en_data]

        self.ja_pad_data = pad_sequence(ja_data, batch_first=True)
        self.en_pad_data = pad_sequence(en_data, batch_first=True)

    def __len__(self):
        return len(self.ja_pad_data)

    def __getitem__(self, idx):
        return self.ja_pad_data[idx], self.ja_length[idx], self.en_pad_data[idx], self.en_length[idx]

emb_dim = 300
hidden_dim = 256
batch_size = 128

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# データの準備
ja_data, en_data = get_data("small_parallel_enja/train.ja", "small_parallel_enja/train.en")
test_ja_data, test_en_data = get_data("small_parallel_enja/test.ja", "small_parallel_enja/test.en")

dataset = S2SDataset(ja_data, en_data)
testdataset = S2SDataset(test_ja_data, test_en_data)

train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(testdataset, batch_size=batch_size, shuffle=True)

model = Seq2SeqRNN(len(ja_word2id), len(en_word2id), emb_dim, hidden_dim)
model.to(device)

optimizer = optim.Adam(model.parameters())

def train(model, train_loader):
    model.train()
    total_loss = 0
    correct_num = 0
    len_loader = len(train_loader)

    for ja_data, ja_len, en_data, en_len in train_loader:
        # GPUへ
        ja_data = ja_data.to(device)
        ja_len = ja_len.to(device)
        en_data = en_data.to(device)
        en_len = en_len.to(device)

        optimizer.zero_grad()
        loss = model(ja_data, ja_len, en_data, en_len)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss/len_loader

def evaluation(model, valid_loader):
    model.eval()
    total_loss = 0
    correct_num = 0
    len_loader = len(valid_loader)
    for ja_data, ja_len, en_data, en_len in valid_loader:
      # GPUへ
      ja_data = ja_data.to(device)
      ja_len = ja_len.to(device)
      en_data = en_data.to(device)
      en_len = en_len.to(device)         

      with torch.no_grad():
        loss = model(ja_data, ja_len, en_data, en_len)
        total_loss += loss.item()

    return total_loss/len_loader

print(valid_loader.__iter__())
for epoch in range(100):
    train_loss = train(model, train_loader)
    valid_loss = evaluation(model, valid_loader)
    print(f"epoch: {epoch}")
    print(f"<train> Loss: {train_loss}")
    print(f"<valid> Loss: {valid_loss}")

# todo validデータから一部データを抽出し，翻訳結果を表示する処理
"""
print(test_en_data.__getitem__(0))
valid_data = test_en_data.__getitem__(0)
sample = model.generate(valid_data,len(valid_data))
print(sample)

"""
# 保存
data_path = '/content/gdrive/My Drive/Colab Notebooks/kobo2020/datarepo11/'#todo ファイル出力ディレクトリを指定
torch.save(model.state_dict(), data_path + "trained_model.pt")


# 発展課題(任意)
# 1：入力した任意の日本語文を英語に翻訳するプログラムの作成(本プログラム内で学習済みモデルを保存し，そのプログラムで読み込む)
# 2：LSTMをGRUに変更
# 3: LSTMをBidirectional LSTM(もしくはBidirectional GRU)へ変更
# 4：Attention付きSeq2Seqへの拡張


epoch: 0
<train> Loss: 0.9577345888671058
<valid> Loss: 0.1255793645977974
epoch: 1
<train> Loss: 0.07774621478336699
<valid> Loss: 0.027657748199999332
epoch: 2
<train> Loss: 0.02024581527951962
<valid> Loss: 0.006380392238497734
epoch: 3
<train> Loss: 0.004842710471652505
<valid> Loss: 0.002281957247760147
epoch: 4
<train> Loss: 0.0018476594383578242
<valid> Loss: 0.0013161762908566743
epoch: 5
<train> Loss: 0.0010925751461414025
<valid> Loss: 0.0008819970535114408
epoch: 6
<train> Loss: 0.0007329589280578525
<valid> Loss: 0.0006174621084937826
epoch: 7
<train> Loss: 0.0005225732471035255
<valid> Loss: 0.0004511200968408957
epoch: 8
<train> Loss: 0.0003865045742304457
<valid> Loss: 0.00034068780223606154
epoch: 9
<train> Loss: 0.000292645963465633
<valid> Loss: 0.00026141144189750776
epoch: 10
<train> Loss: 0.00022567855287253943
<valid> Loss: 0.00020374841915327124
epoch: 11
<train> Loss: 0.0001763667093034324
<valid> Loss: 0.00016037480236263946
epoch: 12
<train> Loss: 0.0001392109

In [81]:
#seq2seq テスト用
import pickle
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence, pad_sequence, pack_padded_sequence

ja_word2id_path = "jaword2id.pickle"
en_word2id_path = "enword2id.pickle"
ja_word2id = pickle.load(open(ja_word2id_path, "rb"))
en_word2id = pickle.load(open(en_word2id_path, "rb"))

#言語に応じたデータを取り出してidのリストを返す
def get_ids(word_list, lang):
    if lang == "ja":
        word2id = ja_word2id
    else:
        word2id = en_word2id

    ids = []
    for word in word_list:
        if word in word2id:
            ids.append(word2id[word])
        else:
            ids.append(word2id["[UNK]"])
    return ids


# 使用するデータの英語と日本語の文に応じたid列のテンソル が入ったリストを返す
def get_data(ja_fname, en_fname):
    ja_ids_list = []
    en_ids_list = []

    with open(ja_fname) as f:
        for line in f:
            if not line:
                continue
            word_list = line.split()
            ids = get_ids(word_list, "ja")
            ja_ids_list.append(torch.tensor(ids))

    with open(en_fname) as f:
        for line in f:
            if not line:
                continue
            word_list = line.split()
            ids = get_ids(word_list, "en")
            en_ids_list.append(torch.tensor(ids))

    return ja_ids_list, en_ids_list

class Seq2SeqRNN(nn.Module):
  
    def __init__(self, ja_vocab_size, en_vocab_size, embedding_dim, hidden_dim):
        super(Seq2SeqRNN, self).__init__()
        self.ja_embedding = nn.Embedding(ja_vocab_size, embedding_dim)
        self.en_embedding = nn.Embedding(en_vocab_size, embedding_dim)
        self.encoder = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.decoder = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, en_vocab_size, bias=True)
        self.crossentropy = nn.CrossEntropyLoss()
        #print(ja_vocab_size)#4905
        #print(en_vocab_size)#4108

    def forward(self, ja_data, ja_data_len, en_data, en_data_len):
            # ja_dataを単語埋め込み
            embed_ja_data = self.ja_embedding(ja_data)
            # embed_ja_dataをPackedSequence型のデータに変換
            x = pack_padded_sequence(embed_ja_data, ja_data_len, batch_first=True, enforce_sorted=False)
            # encoderに入力し，エンコード結果を得る
            _, (h, c) = self.encoder(x)

            # en_dataの各系列の先頭に[BOS]のidを追加したbos_en_dataを作成
            ones = torch.ones(len(en_data_len), 1, dtype=torch.long).to(device)
            bos_en_data = torch.cat([ones, en_data], dim=-1)

            # en_data_lenをその分すべて1ずつ増やす
            # todo
            en_data_len += 1

            # bos_en_dataを単語埋め込み
            embed_bos_en_data = self.en_embedding(en_data)# todo

            # bos_en_dataをpack_padded_sequenceを用いてPackedSequence型のデータに変換
            packed_en_data = pack_padded_sequence(embed_bos_en_data, ja_data_len, batch_first=True, enforce_sorted=False)# todo
            # bos_en_dataのPackedSequenceとja_dataのエンコード結果を用いてデコードし，デコード結果をoutputに代入．
            # todo
            output,(dh,dc) = self.decoder(packed_en_data , (h,c))

            # デコード結果outputを線形層でアフィン変換
            pred = self.linear(output.data)# todo

            # ここからはクロスエントロピー損失計算のための正解の準備
            # en_dataの各系列の末尾に[EOS]のidを追加したeos_en_dataを作成
            zeros = torch.zeros(len(en_data_len), 1, dtype=torch.long).to(device)# todo
            eos_en_data =  torch.cat([en_data,zeros],dim = 1)# todo

            # eos_en_data(埋め込み前)をすでに1増やされたen_data_lenを用いてpack_padded_sequence．これにより単語ID列 + [EOS]のID(0)の系列のPackedSequenceが得られる
            target =  pack_padded_sequence(eos_en_data, ja_data_len, batch_first=True, enforce_sorted=False)# todo
            #print(pred.data)#grad_fnない
            loss = self.crossentropy(pred,target.data)# todo
            #print(loss)
            return loss


     # todo 日本語文を受け取り，英語文を生成するメソッド

    def generate(self, ja_data, ja_data_len):
        sampled = []
        sample_id = ja_data[0]
        ja_data = ja_data.to(device)

        for _ in range(ja_data_len):
            x = torch.tensor([sample_id]).reshape((1, 1)).to(device)

            out = self.en_embedding.forward(x)#埋め込みベクトルに変換
            _,(h,c) = self.encoder.forward(out)#隠れ状態を得る 

            out,(dh,dc) = self.decoder.forward(out,(h,c))#デコーダーに入れる
            score = self.linear(out)#線形変換
            
            sample_id = torch.max(score.flatten(),axis = 0).indices.item()
            sampled.append(sample_id)
      
        return sampled  
"""    
     def generate(self, h, start_id, sample_size):
        sampled = []
        sample_id = start_id
        self.lstm.set_state(h)

        for _ in range(sample_size):
            x = np.array(sample_id).reshape((1, 1))
            out = self.embed.forward(x)
            out = self.lstm.forward(out)
            score = self.affine.forward(out)

            sample_id = np.argmax(score.flatten())
            sampled.append(int(sample_id))

        return sampled          


        ここで実行する手順
        1. エンコーダーLSTM層に日本語の単語埋め込み系列を入力し，各系列の最後の入力を行った時点の隠れ状態+記憶セルを取得
        2. デコーダーLSTM層に先頭に[BOS](単語idは1)を付加した英語の単語埋め込み系列とエンコーダーから得られた隠れ状態+記憶セルを入力し，各単語を入力した際の隠れ状態を得る
        3. 得られた隠れ状態を線形層に入力し，各時点における出力英単語の分布を得る
        4. 末尾に[EOS](単語idは0)を付加した英語の単語系列を正解とし，出力英単語の分布からCrossEntropyLossを計算
        5. CrossEntropyLossを返り値として返す
        具体例：エンコーダーには「これ は ペン です」の単語埋め込み系列を入力し，「です」を入力した時点の隠れ状態+記憶セルを取得．
        　　　　デコーダーには「[BOS] This is a pen」の単語埋め込み系列とエンコーダーから得た隠れ状態+記憶セルを入力し，「This is a pen [EOS]」と出力するように損失を計算
        備考：Teacher Forcingによる学習 (参考：https://satopirka.com/2018/02/encoder-decoder%E3%83%A2%E3%83%87%E3%83%AB%E3%81%A8teacher-forcingscheduled-samplingprofessor-forcing/)
​
        ヒント：
        　・[BOS]と[EOS]の付加について：0のみ，1のみが要素の任意の形状のtensorを作成できるtorch.zeros, torch.onesメソッドを使用して作成したtensorをtorch.catメソッドで単語id系列のtensorとくっつける
        　・デコーダーの出力と正解文の間の損失の計算：PackedSequence型データをLSTMに入力した場合の各時刻の隠れ状態はPackedSequence型で返ってくる．インスタンス変数「.data」でPackedSequenceの持つtensorを得られるが，
        　　　                                    PackedSequenceの作成時に入力したtensorにおける系列の並び・形状と「.data」で得られるtensorの系列の並び・形状は異なる．
        　　　　　　　　　　　　　　　　　　　　　　(参考：sequence.pyのpad_inputsと，pack_padded_sequenceによって得られるxの内容の比較)
                　　　                            そこで，正解文のtensorも一旦PackedSequence化し，tensorの並び・形状を揃えた上で「.data」でtensorを取り出し，CrossEntropyLossを計算する

"""





class S2SDataset(torch.utils.data.Dataset):
    def __init__(self, ja_data, en_data):
        self.ja_length = [len(words) for words in ja_data]
        self.en_length = [len(words) for words in en_data]

        self.ja_pad_data = pad_sequence(ja_data, batch_first=True)
        self.en_pad_data = pad_sequence(en_data, batch_first=True)

    def __len__(self):
        return len(self.ja_pad_data)

    def __getitem__(self, idx):
        return self.ja_pad_data[idx], self.ja_length[idx], self.en_pad_data[idx], self.en_length[idx]

emb_dim = 300
hidden_dim = 256
batch_size = 128

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# データの準備
ja_data, en_data = get_data("small_parallel_enja/train.ja", "small_parallel_enja/train.en")
test_ja_data, test_en_data = get_data("small_parallel_enja/test.ja", "small_parallel_enja/test.en")

dataset = S2SDataset(ja_data, en_data)
testdataset = S2SDataset(test_ja_data, test_en_data)

train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(testdataset, batch_size=batch_size, shuffle=True)

model = Seq2SeqRNN(len(ja_word2id), len(en_word2id), emb_dim, hidden_dim)
model.to(device)

optimizer = optim.Adam(model.parameters())

def train(model, train_loader):
    model.train()
    total_loss = 0
    correct_num = 0
    len_loader = len(train_loader)

    for ja_data, ja_len, en_data, en_len in train_loader:
        # GPUへ
        ja_data = ja_data.to(device)
        ja_len = ja_len.to(device)
        en_data = en_data.to(device)
        en_len = en_len.to(device)

        optimizer.zero_grad()
        loss = model(ja_data, ja_len, en_data, en_len)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss/len_loader

def evaluation(model, valid_loader):
    model.eval()
    total_loss = 0
    correct_num = 0
    len_loader = len(valid_loader)
    for ja_data, ja_len, en_data, en_len in valid_loader:
      # GPUへ
      ja_data = ja_data.to(device)
      ja_len = ja_len.to(device)
      en_data = en_data.to(device)
      en_len = en_len.to(device)         

      with torch.no_grad():
        loss = model(ja_data, ja_len, en_data, en_len)
        total_loss += loss.item()

    return total_loss/len_loader

print(valid_loader.__iter__())
for epoch in range(1):
    train_loss = train(model, train_loader)
    valid_loss = evaluation(model, valid_loader)
    print(f"epoch: {epoch}")
    print(f"<train> Loss: {train_loss}")
    print(f"<valid> Loss: {valid_loss}")

# todo validデータから一部データを抽出し，翻訳結果を表示する処理

print(test_en_data.__getitem__(1))
valid_data = test_en_data.__getitem__(1)
sample = model.generate(valid_data,len(valid_data))
print(sample)


# 保存
"""
data_path = '/content/gdrive/My Drive/Colab Notebooks/kobo2020/datarepo11/'#todo ファイル出力ディレクトリを指定
torch.save(model.state_dict(), data_path + "trained2_model.pt")
"""

# 発展課題(任意)
# 1：入力した任意の日本語文を英語に翻訳するプログラムの作成(本プログラム内で学習済みモデルを保存し，そのプログラムで読み込む)
# 2：LSTMをGRUに変更
# 3: LSTMをBidirectional LSTM(もしくはBidirectional GRU)へ変更
# 4：Attention付きSeq2Seqへの拡張


epoch: 0
<train> Loss: 0.9300591720034704
<valid> Loss: 0.11961556039750576
tensor([  9, 164,  19, 244,  16, 652,   3])


RuntimeError: ignored

In [79]:
import torch
import torch.nn as nn
from torchvision import transforms
 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
 
#====== データセット =======
 
#====== モデル(ネットワーク)　設定======
readedmodel = Seq2SeqRNN(len(ja_word2id), len(en_word2id), emb_dim, hidden_dim)
readedmodel.to(device)
 
#====== ロード =======
data_path = '/content/gdrive/My Drive/Colab Notebooks/kobo2020/datarepo11/'#todo ファイル出力ディレクトリを指定
#読み込み
readedmodel.load_state_dict(torch.load(data_path+ "trained_model.pt", map_location=device))
#model.load_state_dict(torch.load(data_path+ "trained_model.pt"))
# todo validデータから一部データを抽出し，翻訳結果を表示する処理

print(test_en_data.__getitem__(1))
valid_data = test_en_data.__getitem__(1)
sample = model.generate(valid_data,len(valid_data))
print(sample)


tensor([  9, 164,  19, 244,  16, 652,   3])
[9, 9, 9, 9, 9, 9, 9]



１個目の実行結果からlossは減少しているので、学習はできているように思う。ただ、その学習しているモデルを使用できていないのか、generateすると同じ単語IDがずっと出力され続けるようになってしまった。<br>forward作るのに時間がかかりすぎた & このよくわからない出力をするgenerateを作成して時間がなくなってしまいました...。
